In [ ]:
import os, glob
import cv2
import numpy as np
import numpy as np
from matplotlib import pyplot as plt
from qreader import QReader

In [ ]:

token = "" # set a user token
secret = "" # set the green ai token

base_url = "http://xxxx/" # set according to your setup

In [ ]:
# Create a QReader instance
qreader = QReader()
    
def getNumber(filename):
    global qreader

    
    img = cv2.imread(filename)
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    rotated_image = cv2.rotate(image_rgb, cv2.ROTATE_180)
    gray = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2GRAY  )

    # Apply thresholding to separate white regions
    _, thresholded = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    rects = []
    roi = None
    
    # Iterate through the contours
    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
    
        # Check if the polygon has 4 vertices (indicating a rectangle)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            if 500 <= w <= 1000:       
                rects.append(approx)
                roi = gray[y:y + h, x:x + w]
                roi_ts = thresholded[y:y + h, x:x + w]
                
                
                bottom_left, bottom_right, _,_ = sorted(approx, key=lambda x: x[0][1]) # sort by y axis
                
                bottom_left, bottom_right = sorted([bottom_left, bottom_right], key=lambda x: x[0][0]) # sort by x axis
                
                angle = np.arctan2(bottom_right[0][1] - bottom_left[0][1], bottom_right[0][0] - bottom_left[0][0]) * 180 / np.pi
                
                rotated_roi = cv2.warpAffine(roi, cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0), (w, h))


                roi = rotated_roi
               

    foundRect = True
    
    if len(rects) != 1:
        print("rects should be: 1, is",len(rects))

    if len(rects) == 0:
        foundRect = False
        return (None, -1)


    if foundRect:
        print("Found rect")
    
        target_width = 200
        aspect_ratio = roi.shape[1] / roi.shape[0]
        target_height = int(target_width / aspect_ratio)
        
        roi_resized = cv2.resize(roi, (target_width, target_height))
        
        border_size = 2
        roi_without_border = roi_resized[border_size:-border_size, border_size:-border_size]


    
    
        # Get the image that contains the QR code
        #image = cv2.cvtColor(cv2.imread("path/to/image.png"), cv2.COLOR_BGR2RGB)
        
        # Use the detect_and_decode function to get the decoded QR data
        decoded_text = qreader.detect_and_decode(image=roi_without_border)
        #print(decoded_text[0])
        
        return (roi_without_border, decoded_text[0])    
    else:
        print("did not find rect -  brute force")
        decoded_text = qreader.detect_and_decode(image=gray)
        #print(decoded_text[0])
        
        return (gray, decoded_text[0])  
    

In [ ]:
filenames = []

filenames_wo_number = []

import os, shutil
dest_path_ok = os.path.join("scans","result","numbered")
dest_path_err = os.path.join("scans","result","err")
os.makedirs(dest_path_ok, exist_ok=True)
os.makedirs(dest_path_err, exist_ok=True)

In [ ]:

for (i,f) in enumerate( glob.glob(os.path.join(".\\scans\\DOC*\\","*.jpg"))): # modify according to your setup
    print("################### START ###############")
    
    print("%s %s" % (i,f))
    
    try:
        (roi, number) = getNumber(f)

        number = int(number)

        if number < 1:
            raise Exception("shall not be smaller 1")
    
    except Exception as e:
        print("E: %s" % e)
        
        filenames_wo_number.append(f)
        shutil.copyfile(f, os.path.join(dest_path_err, "E %s.jpg" % i))
        continue
        pass

    print("######")
    print("#");
    print("## %s" % number)
    print("#");
    print("######")

    

    try:
        font = cv2.FONT_HERSHEY_SIMPLEX
        position = (130, 30)  # (x, y) coordinates where the text will be placed
        font_scale = 1  # Font scale factor
        font_color = (0, 0, 0)  # Font color in BGR format (Blue, Green, Red)
        thickness = 2  # Thickness of the text

        cv2.putText(roi, str(number), position, font, font_scale, font_color, thickness)
        
        plt.imshow(roi, cmap='gray')
        plt.title('my picture')
        plt.show()

        # copy file
        shutil.copyfile(f, os.path.join(dest_path_ok, "%s.jpg" % number))
    except Exception as e:
        #raise e
        shutil.copyfile(f, os.path.join(dest_path_err, "E %s.jpg" % i))
        continue

    print("################### END ###############")

In [ ]:
auth_headers = {
    'Authorization': f'Token {token}',  # Assuming Token-based authentication
    'GreenAiToken': secret
}

headers = {
    'Authorization': auth_headers['Authorization'],
    'GreenAiToken': secret,
    'Content-Type': 'application/json',
}

In [ ]:
# client.py
import requests, os, hashlib
from tqdm.utils import CallbackIOWrapper
from tqdm import tqdm

def upload_file(file_path, participant_id, headers=None):
    url = base_url+'api/uploadedfile/create_upload_file/' # set the url here

    
    params = {'greenai_id': participant_id,'upload_type':"foam"}
    response = requests.post(url, params=params, headers=headers)
    if response.status_code == 201:
        print("File uploaded successfully!")
    else:
        print("Failed to upload file:", response.text)

    print(response.json())

    


    with open(file_path, 'rb') as file:
        file.seek(0)
        checksum = hashlib.sha256(file.read()).hexdigest()
        file.seek(0)
    
        print("Checksum: %s" % checksum)
        # upload the file
    
        headersWithContent = headers
        #headersWithContent['Content-Disposition'] = 'attachment; filename='+localFile.file.name
        #headersWithContent['Content-Type'] =  "application/octet-stream"
    
       
        #files = {'file': (localFile.file.name, file, 'multipart/form-data')}
        url = base_url+"link-file-to-uploaded-file/%s/" % response.json()['id']
    
        try:
            file_size = os.stat(file_path).st_size
            with tqdm(total=file_size, unit="B", unit_scale=True, unit_divisor=1024) as t:
                wrapped_file = CallbackIOWrapper(t.update, file, "read")                
                files = {'file': wrapped_file}
                uploadResponse = requests.post(url, files=files, headers=auth_headers)
                
        except requests.RequestException as e:
            print(f"Request error: {e}")
            return
    
    
        print(uploadResponse.text)




In [ ]:
# upload
import requests, glob



for f in glob.glob(os.path.join(dest_path_ok, "*.jpg")):
    print(f)
    i = os.path.basename(f).split(".")[0]
    try:
        upload_file(f, participant_id=i, headers=headers)
        print(i)
    except:
        print("Could not upload %s" % f)


    

